# Projet

## Partie 1 : Perceptron

**Indiquer et expliquer la taille de chaque tenseur dans le fichier perceptron pytorch.py fourni.**

- $w$ est de taille $784 \times 10$. $784$ est le nombre de pixels dans chaque image ($28 \times 28$) et donc la taille de l'entrée et $10$ est la taille des vecteurs de label et de la sortie.  
- $b$ est de taille $1 \times 10$ car c'est le biais qui est de même dimension que la sortie.  
- $data\_train$ est de taille $784 \times nb\_data\_train$, $nb\_data\_train$ est le nombre d'images dans le set d'entrainement.  
- $data\_test$ est de taille $784 \times nb\_data\_test$, $nb\_data\_test$ est le nombre d'images dans le set de test.  
- $x$ est un batch de $data\_train$ ($784 \times batch\_size$)
- $y$ est la sortie associé au batch $x$ ($10 \times batch\_size$)
- $t$ est un batch des labels, associés à $x$ ($10 \times batch\_size$)

## Partie 2 : Shallow network

Implémentation de l’algorithme du perceptron multi-couches avec une seule couche cachée et une sortie linéaire.  
L'objectif est de trouver les hyperparamètres $\eta$ et le nombre de neurones de la couche cachée ($N$).

### Méthodologie

Le $\eta$ par défaut nous paraissait bien trop petit, nous avons observé que même après 10 epochs le perceptron continuait d'apprendre. Nous avons augmenté sa valeur à : $\eta = 0.001$. Cette valeur nous aprait correcte car l'accuracy de notre modèle se stabilise autours des 96% pour un $N = 100$.  

### Influence des paramètres sur la performance

$\eta$ à une influence sur la vitesse d'apprentissage. Si on se limite à 10 epochs l'apprentissage doit être assez rapide pour obtenir des résultats satisfaisants.

$N$ influence la précision du résultat, plus il y a de neurones plus la probabilité de trouver le bon résultat augmente mais aussi plus l'apprentissage est lent. En effet, plus de neurones signifie plus de calculs donc plus de temps d'apprentissage.
